<a href="https://colab.research.google.com/github/madeyemocki/xai_fnd_data/blob/main/Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import json
import pandas as pd
import requests
import io
from urllib.request import urlopen
import json
import logging
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from pandas.io.json import json_normalize
from pprint import pprint
from scipy.stats import shapiro


In [ ]:
# installing mne for the BH correction
!pip install mne
import mne

!pip install scikit_posthocs
import scikit_posthocs as sp

!pip install statannotations

!pip install pingouin
import pingouin as pg

In [3]:
#Ausgangs Dataframes 
df_v1 = pd.read_csv('data/v1_cleaned.csv')
df_v2 = pd.read_csv('data/v2_cleaned.csv')

# merge the dataframes
df_all_versions = pd.concat([df_v1, df_v2], ignore_index=True)

In [ ]:
#check
columns = list(df_v1.columns)
pprint(columns)

# Demographics
in prozent?

In [13]:
# get columns which start with "demographics"
demographics_cols = [col for col in df_all_versions.columns if col.startswith('demographics')]
demographics_cols


# country and nationality are not relevant, since the study was conducted in the US

['demographics.age',
 'demographics.country',
 'demographics.education',
 'demographics.employment',
 'demographics.gender',
 'demographics.income',
 'demographics.nationality']

In [ ]:
for col in demographics_cols:
    value_counts = df_all_versions[col].value_counts()
    print(f"Spalte: {col}")
    print(value_counts)
    print("\n")

# Usa, USA... zusammenfassen?

# Information Literacy
for each version?

In [41]:
information_literacy_cols = [col for col in df_all_versions.columns if col.startswith('information-literacy')]
information_literacy_cols

# Extrahiere die gemeinsamen Spalten
common_cols_v1 = df_v1.columns.intersection(information_literacy_cols)

# Erstelle ein neues DataFrame mit den gemeinsamen Spalten
df_v1_information_literacy = df_v1[common_cols_v1].copy()

# Extrahiere die gemeinsamen Spalten
common_cols_v2 = df_v2.columns.intersection(information_literacy_cols)

# Erstelle ein neues DataFrame mit den gemeinsamen Spalten
df_v2_information_literacy = df_v2[common_cols_v1].copy()



In [42]:
#Chronbachs Alpha reliability of prot behav
prot_beh = df_v1[['information-literacy.access-relevant-literature',
       'information-literacy.check-reliability',
       'information-literacy.differentiate-right-and-fake',
       'information-literacy.efficiently-use-subscribed-and-openaccess',
       'information-literacy.have-it-skills',
       'information-literacy.identify-authentic-sources',
       'information-literacy.identify-misinformation',
       'information-literacy.plagiarism-and-publication-ethics',
       'information-literacy.research-level-skilss']]
pg.cronbach_alpha(data=prot_beh) 
#first position is cronbachs alpha, second is confidence interval 95% (0.9 excellent, 0.8 good, 0.7 accaptable)

(0.9269078654422946, array([0.907, 0.944]))

In [43]:
#Chronbachs Alpha reliability of prot behav
prot_beh = df_v2[['information-literacy.access-relevant-literature',
       'information-literacy.check-reliability',
       'information-literacy.differentiate-right-and-fake',
       'information-literacy.efficiently-use-subscribed-and-openaccess',
       'information-literacy.have-it-skills',
       'information-literacy.identify-authentic-sources',
       'information-literacy.identify-misinformation',
       'information-literacy.plagiarism-and-publication-ethics',
       'information-literacy.research-level-skilss']]
pg.cronbach_alpha(data=prot_beh) 

(0.8761898427566921, array([0.838, 0.909]))

In [44]:
#Normalverteilung

# Calculate Shapiro-Wilk, mean, and median for each column
results = []
for column in df_v1_information_literacy.columns:
    shapiro_test = shapiro(df_v1_information_literacy[column])
    mean = df_v1_information_literacy[column].mean()
    median = df_v1_information_literacy[column].median()
    results.append((column, shapiro_test.statistic, shapiro_test.pvalue, mean, median))

# Create a DataFrame with the results
df_results = pd.DataFrame(results, columns=['Column', 'Shapiro-Wilk', 'p-value', 'Mean', 'Median'])

# Display the DataFrame
df_results
# If the chosen alpha level is 0.05 and the p-value is less than 0.05, then the null hypothesis that the data are normally distributed is rejected
# p-value bei Shapiro wenn unter 0.05 dann keine Normalverteilung dann nicht parametrische tests
# the data in these columns do not follow a normal distribution. 
# Information Literacy ins zusammenhang mit fp tn erkennung bringen, was wenn bei einem parametrische tests gemacht werden sollen und beim andern parmetrische?
# Wann part wo ich nochmal p value messe?



,Column,Shapiro-Wilk,p-value,Mean,Median
0,information-literacy.access-relevant-literature,0.799605,1.167457e-12,5.964789,6.0
1,information-literacy.check-reliability,0.860818,3.029834e-10,5.830986,6.0
2,information-literacy.differentiate-right-and-fake,0.867269,5.949396e-10,5.739437,6.0
3,information-literacy.efficiently-use-subscribe...,0.846471,7.244923e-11,5.676056,6.0
4,information-literacy.have-it-skills,0.903314,4.040287e-08,5.042254,5.0
5,information-literacy.identify-authentic-sources,0.840797,4.215798e-11,5.661972,6.0
6,information-literacy.identify-misinformation,0.870772,8.660047e-10,5.380282,5.5
7,information-literacy.plagiarism-and-publicatio...,0.802334,1.456426e-12,5.887324,6.0
8,information-literacy.research-level-skilss,0.843617,5.508796e-11,5.718310,6.0


In [45]:
#Normalverteilung

# Calculate Shapiro-Wilk, mean, and median for each column
results = []
for column in df_v2_information_literacy.columns:
    shapiro_test = shapiro(df_v2_information_literacy[column])
    mean = df_v2_information_literacy[column].mean()
    median = df_v2_information_literacy[column].median()
    results.append((column, shapiro_test.statistic, shapiro_test.pvalue, mean, median))

# Create a DataFrame with the results
df_results = pd.DataFrame(results, columns=['Column', 'Shapiro-Wilk', 'p-value', 'Mean', 'Median'])

# Display the DataFrame
df_results
# If the chosen alpha level is 0.05 and the p-value is less than 0.05, then the null hypothesis that the data are normally distributed is rejected
# p-value bei Shapiro wenn unter 0.05 dann keine Normalverteilung dann nicht parametrische tests
# the data in these columns do not follow a normal distribution. 
# Information Literacy ins zusammenhang mit fp tn erkennung bringen, was wenn bei einem parametrische tests gemacht werden sollen und beim andern parmetrische?
# Wann part wo ich nochmal p value messe?



,Column,Shapiro-Wilk,p-value,Mean,Median
0,information-literacy.access-relevant-literature,0.750139,3.318344e-12,6.074766,6.0
1,information-literacy.check-reliability,0.830571,9.847583e-10,6.084112,6.0
2,information-literacy.differentiate-right-and-fake,0.855474,8.195029e-09,5.766355,6.0
3,information-literacy.efficiently-use-subscribe...,0.870286,3.229915e-08,5.747664,6.0
4,information-literacy.have-it-skills,0.904524,1.160630e-06,4.953271,5.0
5,information-literacy.identify-authentic-sources,0.865840,2.119202e-08,5.813084,6.0
6,information-literacy.identify-misinformation,0.867799,2.548819e-08,5.514019,6.0
7,information-literacy.plagiarism-and-publicatio...,0.791589,5.187702e-11,5.925234,6.0
8,information-literacy.research-level-skilss,0.825391,6.501285e-10,5.869159,6.0


In [ ]:
#todo v3

# Information seeking behavior

In [ ]:
columns = list(df_all_versions.columns)
pprint(columns)

In [48]:
information_seeking_behavior_cols = [col for col in df_all_versions.columns if col.startswith('information-seeking')]
information_seeking_behavior_cols

# Extrahiere die gemeinsamen Spalten
common_cols_v1 = df_v1.columns.intersection(information_seeking_behavior_cols)

# Erstelle ein neues DataFrame mit den gemeinsamen Spalten
df_v1_information_seeking_behavior = df_v1[common_cols_v1].copy()

# Extrahiere die gemeinsamen Spalten
common_cols_v2 = df_v2.columns.intersection(information_seeking_behavior_cols)

# Erstelle ein neues DataFrame mit den gemeinsamen Spalten
df_v2_information_seeking_behavior = df_v2[common_cols_v2].copy()

df_v2_information_seeking_behavior


,information-seeking.access.google,information-seeking.access.library,information-seeking.access.personal-purchase,information-seeking.access.websites,information-seeking.accessibility-of-information-about-transparency,information-seeking.comprehensive-information,information-seeking.most-used-sources.0,information-seeking.most-used-sources.1,information-seeking.most-used-sources.2,information-seeking.purpose.general-awareness,...,information-seeking.sources.newspaper,information-seeking.sources.online-newspaper,information-seeking.sources.personal-social-networks,information-seeking.sources.twitter,information-seeking.top-source-seriousness,information-seeking.truthfulness.facebook,information-seeking.truthfulness.instagram,information-seeking.truthfulness.newspaper,information-seeking.truthfulness.online-newspaper,information-seeking.truthfulness.twitter
0,7,7,4,7,5,7,online-newspaper,instagram,facebook,7,...,4,6,7,1,6,4,5,6,6,4
1,7,5,2,6,5,7,blogs-websites,personal-social-networks,facebook,6,...,2,2,4,2,5,4,4,5,5,4
2,7,4,6,5,7,7,facebook,online-newspaper,newspaper,6,...,5,6,1,3,5,3,4,7,5,4
3,5,5,4,5,5,5,online-newspaper,newspaper,blogs-websites,5,...,5,5,2,1,6,2,3,5,5,2
4,6,4,4,6,5,7,online-newspaper,twitter,blogs-websites,6,...,3,5,2,5,5,5,5,6,6,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,3,1,1,3,5,5,online-newspaper,personal-social-networks,newspaper,6,...,1,2,4,1,5,2,3,5,5,3
103,6,5,6,6,6,7,online-newspaper,blogs-websites,twitter,7,...,7,7,7,7,6,4,4,5,5,4
104,6,1,1,6,5,6,blogs-websites,personal-social-networks,facebook,7,...,2,3,5,4,5,4,4,5,5,4
105,6,1,4,6,7,7,blogs-websites,personal-social-networks,newspaper,7,...,1,6,6,1,7,1,1,7,7,1


In [50]:
access_columns = [col for col in df_v1_information_seeking_behavior.columns if col.startswith('information-seeking.access')]
access_columns

['information-seeking.access.google',
 'information-seeking.access.library',
 'information-seeking.access.personal-purchase',
 'information-seeking.access.websites',
 'information-seeking.accessibility-of-information-about-transparency']